Using Gower Distance and DBSCAN to cluster customers of age 10-30

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gower
from sklearn.cluster import DBSCAN

In [ ]:
# Column Names are changed manually
table_nm = 'DLAB' 
names = ['iqr_dt','cus_no','act_no','mkt_gb','pdt_gb','iem_cd','iem_krl_anm','bnc_qty','tot_aet_amt'] 
file_path = f'/home/{table_nm}_1030.csv' 
df_trd_1 = pd.read_csv(file_path)

table_nm = 'DLAB2' 
names = ['iqr_dt','cus_no','act_no','mkt_gb','pdt_gb','iem_cd','iem_krl_anm','bnc_qty','tot_aet_amt'] 
file_path = f'/home/{table_nm}2_1030.csv' 
df_trd_2 = pd.read_csv(file_path) 

table_nm = 'DLAB3' 
names = ['cus_no','sex_dit_cd','cus_age','zip_ctp_cd', 'tco_cus_grd_cd', 'ivs_icn_cd'] 
file_path = f'/home/{table_nm}_1030.csv' 
df_cus = pd.read_csv(file_path) 

table_nm = 'DLAB4' 
names = ['act_no','cus_no','act_opn_ym'] 
file_path = f'/home/{table_nm}_1030.csv' 
df_act = pd.read_csv(file_path) 

In [ ]:
'''
Index(['iqr_dt', 'cus_no', 'act_no', 'mkt_gb', 'pdt_gb', 'iem_cd', iem_krl_anm', 'bnc_qty', 'tot_aet_amt'])
날짜, 고객번호, 계죄번호, 국내/해외, 주식/ETF, 종목코드, 종목한글명, 수량, 총평가액
'''
df_trd = pd.merge(left=df_trd_1, right=df_trd_2[df_trd_2['iqr_dt'] > 20210701], how='outer')
df_trd = df_trd.drop('Unnamed: 0', axis=1)
df_trd.shape 

In [ ]:
'''
Index(['act_no', 'cus_no', 'act_opn_ym'])
계좌번호, 고객번호, 계좌개설년월
'''
df_act = df_act.drop('Unnamed: 0', axis=1)
df_act.shape

In [ ]:
'''
Index(['cus_no', 'sex_dit_cd', 'cus_age', 'zip_ctp_cd', 'tco_cus_grd_cd', 'ivs_icn_cd'])
고객번호, 성별, 나이, 지역, 고객등급코드, 투자성향응답
'''
df_cus = df_cus.drop('Unnamed: 0', axis=1)
df_cus.shape 

In [6]:
'''
choose 440 customers with trd data
remove noisy data (customers with DIREXION SEMICONDUCTOR BULL 3X ETF)
'''
df_cus_w = df_cus.copy()[(df_cus['cus_no'].isin(df_trd['cus_no'])) 
                         & ~(df_cus['cus_no'].isin(df_trd[(df_trd['iem_cd'] == "US25459W4583")]['cus_no']))]
df_cus_w.shape

(493, 6)

In [7]:
'''
For each date from Jan. 1st to Dec. 31st,
sum up each customer's total asset amount 
'''
df_bal = df_trd.copy()[['iqr_dt', 'cus_no', 'tot_aet_amt']].groupby(['iqr_dt', 'cus_no']).sum()
df_bal = df_bal.reset_index()
df_bal.shape 

(144907, 3)

In [8]:
'''
For each date from Jan. 1st to Dec. 31st,
sum up each customer's total [Korean Stock] asset amount
'''
df_bal_kor_stk = df_trd[(df_trd['mkt_gb'] == '국내') 
                        & (df_trd['pdt_gb'] == '주식')].copy()[['iqr_dt', 
                                                              'cus_no', 
                                                              'tot_aet_amt']].groupby(['iqr_dt', 
                                                                                       'cus_no']).sum()
df_bal_kor_stk = df_bal_kor_stk.reset_index()

In [9]:
'''
For each date from Jan. 1st to Dec. 31st,
sum up each customer's total [Foreign Stock] asset amount
'''
df_bal_ovs_stk = df_trd[(df_trd['mkt_gb'] == '해외') 
                        & (df_trd['pdt_gb'] == '주식')].copy()[['iqr_dt', 
                                                              'cus_no', 
                                                              'tot_aet_amt']].groupby(['iqr_dt', 
                                                                                       'cus_no']).sum()
df_bal_ovs_stk = df_bal_ovs_stk.reset_index()

In [10]:
'''
For each date from Jan. 1st to Dec. 31st,
sum up each customer's total [Korean ETF] asset amount
'''
df_bal_kor_etf = df_trd[(df_trd['mkt_gb'] == '국내') 
                        & (df_trd['pdt_gb'] == 'ETF')].copy()[['iqr_dt', 
                                                               'cus_no', 
                                                               'tot_aet_amt']].groupby(['iqr_dt', 
                                                                                        'cus_no']).sum()
df_bal_kor_etf = df_bal_kor_etf.reset_index()

In [11]:
'''
For each date from Jan. 1st to Dec. 31st,
sum up each customer's total [Foreign ETF] asset amount
'''
df_bal_ovs_etf = df_trd[(df_trd['mkt_gb'] == '해외') 
                        & (df_trd['pdt_gb'] == 'ETF')].copy()[['iqr_dt', 
                                                               'cus_no', 
                                                               'tot_aet_amt']].groupby(['iqr_dt', 
                                                                                        'cus_no']).sum()
df_bal_ovs_etf = df_bal_ovs_etf.reset_index()

In [12]:
'''
Combine all the previously calculated columns
'''
df_bal_tot = pd.merge(left=df_bal, 
                      right=df_bal_kor_stk[['iqr_dt','cus_no', 'tot_aet_amt']], 
                      on=['cus_no', 'iqr_dt'], 
                      suffixes=('', '_kor_stk'), 
                      how='left')
df_bal_tot = pd.merge(left=df_bal_tot, 
                      right=df_bal_ovs_stk[['iqr_dt','cus_no', 'tot_aet_amt']], 
                      on=['cus_no', 'iqr_dt'], 
                      suffixes=('', '_ovs_stk'), 
                      how='left')
df_bal_tot = pd.merge(left=df_bal_tot, 
                      right=df_bal_kor_etf[['iqr_dt','cus_no', 'tot_aet_amt']], 
                      on=['cus_no', 'iqr_dt'], 
                      suffixes=('', '_kor_etf'), 
                      
                      how='left')
df_bal_tot = pd.merge(left=df_bal_tot, 
                      right=df_bal_ovs_etf[['iqr_dt','cus_no', 'tot_aet_amt']], 
                      on=['cus_no', 'iqr_dt'], 
                      suffixes=('', '_ovs_etf'), 
                      how='left')
df_bal_tot.shape #(144907, 7)

(144907, 7)

In [13]:
'''
Fill NaN values in merged columns with 0
'''
df_bal_tot[['tot_aet_amt_kor_stk', 
            'tot_aet_amt_ovs_stk', 
            'tot_aet_amt_kor_etf', 
            'tot_aet_amt_ovs_etf']] = df_bal_tot[['tot_aet_amt_kor_stk', 
                                                  'tot_aet_amt_ovs_stk', 
                                                  'tot_aet_amt_kor_etf', 
                                                  'tot_aet_amt_ovs_etf']].fillna(0)

In [14]:
'''
Break the dataframe into a list (by each date)
Create sub-columns that depend on other columns
'''
df_bal_list = []
for i in df_bal_tot['iqr_dt'].unique():
    df = df_bal_tot[df_bal_tot['iqr_dt'] == i]
    df = pd.merge(left=df_cus_w, right=df, on='cus_no', suffixes=('',''))
    df['kor_etf_ratio'] = df['tot_aet_amt_kor_etf'] / (df['tot_aet_amt_kor_stk']+df['tot_aet_amt_kor_etf'])
    df['ovs_etf_ratio'] = df['tot_aet_amt_ovs_etf'] / (df['tot_aet_amt_ovs_stk']+df['tot_aet_amt_ovs_etf'])
    df['kor_ovs_amt_ratio'] = (df['tot_aet_amt_ovs_stk']+df['tot_aet_amt_ovs_etf'] )/ df['tot_aet_amt']
    df[['kor_etf_ratio', 'ovs_etf_ratio', 'kor_ovs_amt_ratio']] = df[['kor_etf_ratio', 'ovs_etf_ratio', 'kor_ovs_amt_ratio']].fillna(0)
    df_bal_list.append(df)

In [15]:
'''
To cluster customers between 10 to 30 years old,
analyze their account info as of Dec. 31, 2021
'''
ten_thirty = df_bal_list[364].set_index(keys='cus_no')
ten_thirty = ten_thirty.drop(['ivs_icn_cd', 'iqr_dt', 'zip_ctp_cd', 'tco_cus_grd_cd'], axis=1) #drop unused columns
ten_thirty ['sex_dit_cd'] = ten_thirty['sex_dit_cd'].replace({1:'M', 2:'F'}) #categorize 

#Uncommenting the lines below will log the asset values 
ten_thirty[['tot_aet_amt_kor_stk',
            'tot_aet_amt_ovs_stk',
            'tot_aet_amt_kor_etf',
            'tot_aet_amt_ovs_etf']] = np.log10(ten_thirty[['tot_aet_amt_kor_stk',
                                                        'tot_aet_amt_ovs_stk',
                                                        'tot_aet_amt_kor_etf',
                                                        'tot_aet_amt_ovs_etf']],
                                             where = (ten_thirty[['tot_aet_amt_kor_stk',
                                                        'tot_aet_amt_ovs_stk',
                                                        'tot_aet_amt_kor_etf',
                                                        'tot_aet_amt_ovs_etf']] != 0))
ten_thirty.shape #(492, 10)

(492, 10)

In [16]:
'''
import 'gower' library
calculate the gower distance between each customer based on selected columns
save the distance info into a matrix

'''
dist_mat = pd.DataFrame(gower.gower_matrix(ten_thirty[['cus_age',
                                                      'tot_aet_amt_kor_stk',
                                                      'tot_aet_amt_ovs_stk',
                                                      'tot_aet_amt_kor_etf',
                                                      'tot_aet_amt_ovs_etf',
                                                      ]]), index=ten_thirty .index, columns=ten_thirty.index)
dist_mat.shape #(492, 492)

(492, 492)

In [17]:
'''
define DBSCAM model with parameters
because dist_mat is already calculated, use metric='precomputed'
fit the model
'''
dbscan_cluster = DBSCAN(eps=0.09, min_samples=20, metric='precomputed')
dbscan_cluster.fit(dist_mat)

DBSCAN(algorithm='auto', eps=0.09, leaf_size=30, metric='precomputed',
       metric_params=None, min_samples=20, n_jobs=None, p=None)

In [ ]:
'''
group labels to count the number of data points in each cluster
'''
ten_thirty ['cluster'] = dbscan_cluster.labels_
ten_thirty_grp = ten_thirty.groupby('cluster').count()
ten_thirty_grp

In [ ]:
'''
briefyly get the idea of how data points are being clustered
'''
ten_thirty.groupby('cluster').mean()

In [ ]:
'''
re-cluster the outliers with a higher eps value
'''
ten_thirty_x = ten_thirty.copy()[ten_thirty['cluster'] == -1]
dist_mat_x = pd.DataFrame(gower.gower_matrix(ten_thirty_x), index=ten_thirty_x .index, columns=ten_thirty_x.index)
dbscan_cluster_x = DBSCAN(eps=0.2, min_samples=15, metric='precomputed')
dbscan_cluster_x.fit(dist_mat_x)

In [ ]:
ten_thirty_x['cluster'] = dbscan_cluster_x.labels_
ten_thirty_x_grp = ten_thirty_x.groupby('cluster').count()
ten_thirty_x_grp

In [ ]:
ten_thirty_x.groupby('cluster').mean()

In [23]:
'''
For each cluster, print out the top five popular stocks on Dec. 31, 2021
'''

def top_five(i):
    return df_trd[(df_trd['iqr_dt'] == 20211231) 
           & (df_trd['cus_no'].isin(ten_thirty[ten_thirty['cluster'] == i-1].reset_index()['cus_no']))].groupby('iem_krl_anm').count().sort_values(by='iqr_dt', 
                                                                                                                                                 ascending=False)['cus_no'].head() / ten_thirty_grp['cus_age'][i-1]

In [ ]:
'''
outliers of outlier group
'''
i = 0
df_trd[(df_trd['iqr_dt'] == 20211231) 
           & (df_trd['cus_no'].isin(ten_thirty_x[ten_thirty_x['cluster'] == i-1].reset_index()['cus_no']))].groupby('iem_krl_anm').count().sort_values(by='iqr_dt', 
                                                                                                                                                 ascending=False)['cus_no'].head() / ten_thirty_x_grp['cus_age'][i-1]

In [ ]:
'''
cluster 0 of outlier group
'''
i = 1
df_trd[(df_trd['iqr_dt'] == 20211231) 
           & (df_trd['cus_no'].isin(ten_thirty_x[ten_thirty_x['cluster'] == i-1].reset_index()['cus_no']))].groupby('iem_krl_anm').count().sort_values(by='iqr_dt', 
                                                                                                                                                 ascending=False)['cus_no'].head() / ten_thirty_x_grp['cus_age'][i-1]

In [ ]:
'''
for each cluster, print out the top 5 popular stocks
'''
for i in range(len(ten_thirty_grp)):
    print('Group:' + str(i-1))
    display(top_five(i))

Below is to show how hierarchical DBSCAN clustering works 

In [27]:
ten_thirty_db = ten_thirty.copy()[['tot_aet_amt',
                                                          'kor_ovs_amt_ratio'
                                                      ]]
dist_mat_db = pd.DataFrame(gower.gower_matrix(ten_thirty_db), index=ten_thirty_db.index, columns=ten_thirty_db.index)

In [ ]:
dbscan_cluster_db = DBSCAN(eps=0.09, min_samples=20, metric='precomputed')
dbscan_cluster_db.fit(dist_mat_db)

ten_thirty_db['cluster'] = dbscan_cluster_db.labels_
ten_thirty_grp_db = ten_thirty_db.groupby('cluster').count()
ten_thirty_grp_db

In [ ]:
cmap = plt.cm.get_cmap("Spectral")
colors = cmap((ten_thirty_db['cluster']+1) / 7)
plt.scatter(ten_thirty_db['tot_aet_amt'], ten_thirty_db['kor_ovs_amt_ratio'], c=colors)
plt.show()

In [ ]:
ten_thirty_db_0 = ten_thirty_db[ten_thirty_db['cluster'] == 0].copy()
dist_mat_db_0 = pd.DataFrame(gower.gower_matrix(ten_thirty_db_0), 
                             index=ten_thirty_db_0.index, 
                             columns=ten_thirty_db_0.index)
dbscan_cluster_db_0 = DBSCAN(eps=0.05, min_samples=2, metric='precomputed')
dbscan_cluster_db_0.fit(dist_mat_db_0)

ten_thirty_db_0['cluster'] = dbscan_cluster_db_0.labels_
ten_thirty_grp_db_0 = ten_thirty_db_0.groupby('cluster').count()
cmap = plt.cm.get_cmap("Spectral")
colors = cmap((ten_thirty_db_0['cluster']+1) / 3.5)
plt.scatter(ten_thirty_db_0['tot_aet_amt'], ten_thirty_db_0['kor_ovs_amt_ratio'], c=colors)
plt.show()

In [ ]:
ten_thirty_db_0 = ten_thirty_db[ten_thirty_db['cluster'] == -1].copy()
dist_mat_db_0 = pd.DataFrame(gower.gower_matrix(ten_thirty_db_0), 
                             index=ten_thirty_db_0.index, 
                             columns=ten_thirty_db_0.index)
dbscan_cluster_db_0 = DBSCAN(eps=0.2, min_samples=2, metric='precomputed')
dbscan_cluster_db_0.fit(dist_mat_db_0)

ten_thirty_db_0['cluster'] = dbscan_cluster_db_0.labels_
ten_thirty_grp_db_0 = ten_thirty_db_0.groupby('cluster').count()
cmap = plt.cm.get_cmap("Spectral")
colors = cmap((ten_thirty_db_0['cluster']+1) / 1.2)
plt.scatter(ten_thirty_db_0['tot_aet_amt'], ten_thirty_db_0['kor_ovs_amt_ratio'], c=colors)
plt.show()